In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd
from numpy import e


In [39]:
# Set path Selenium
CHROMEDRIVER_PATH = '/usr/local/bin/chromedriver'

In [85]:


def job_posts(number):
    driver = webdriver.Chrome()
    links = []
    position_names = []
    companies = []
    locations = []  
    
    path1 = f'https://www.linkedin.com/jobs/search?keywords=Data%20Scientist&location=United%20States'
    driver.get(path1)
    if len(links) %20 == 0:
        batch = len(links) / 20
        print(f'Scraping page number - {20*batch}')
    i = 0
    
    while i < number:
        html = driver.find_element(By.TAG_NAME,'html')
        html.send_keys(Keys.END)
        i = i+1
        try:
            t = driver.find_element(By.XPATH,'//*[@id="main-content"]/section[2]/button')
            t.click()
            time.sleep(2)
        except:
            pass
            time.sleep(3)

            
    
    block = driver.find_element(By.XPATH,'//*[@id="main-content"]/section[2]/ul')
    job_list = block.find_elements(By.CLASS_NAME, 'base-card')
    print(len(job_list))
    
    try:

        for job in job_list:
            try:
                link = job.find_element(By.CLASS_NAME, 'base-card__full-link').get_attribute('href')
                links.append(link)
            except:
                links.append(None)
                ValueError('no link')
            try:
                position_name = job.find_element(By.TAG_NAME,'span').text
                position_names.append(position_name)
            except:
                position_names.append(None)
                ValueError('no pos name')
            try:
                company = job.find_element(By.CLASS_NAME,'hidden-nested-link').text
                companies.append(company)
            except:
                companies.append(None)
                ValueError('no company')
            try:
                location = job.find_element(By.CLASS_NAME,'job-search-card__location').text
                locations.append(location)
            except:
                locations.append(None)
                ValueError('no location')

    except:
        e = 'no path'
        ValueError(e)
        print(e)


    data = pd.DataFrame({'Position':position_names,
            'Company':companies,
            'Location':locations,
            'Link':links})

        
    return data

In [95]:
dataframe=job_posts(50)
dataframe.to_csv('/home/jess/Documents/linked_links.csv')

Scraping page number - 0.0
1000


In [97]:
dataframe

,Position,Company,Location,Link
0,"Data Scientist I, Risk Analytics",Oscar Health,"New York, NY",https://www.linkedin.com/jobs/view/data-scient...
1,"Data Scientist I, Risk Analytics",Oscar Health,United States,https://www.linkedin.com/jobs/view/data-scient...
2,Data Scientist,Sand Cherry,"Denver, CO",https://www.linkedin.com/jobs/view/data-scient...
3,Data Scientist/Data Visualization Engineer - A...,Apple,"Austin, TX",https://www.linkedin.com/jobs/view/data-scient...
4,Data Scientist,TikTok,"Mountain View, CA",https://www.linkedin.com/jobs/view/data-scient...
...,...,...,...,...
995,Lead Data Scientist,S&P Global,United States,https://www.linkedin.com/jobs/view/lead-data-s...
996,"Software Engineer, Machine Learning - Personal...",Grammarly,"Illinois, United States",https://www.linkedin.com/jobs/view/software-en...
997,HR Data Scientist - HRIS Brooklyn Heights,Metro Health,"Brooklyn Heights, OH",https://www.linkedin.com/jobs/view/hr-data-sci...
998,"Software Engineer, Machine Learning Platform",Grammarly,"Illinois, United States",https://www.linkedin.com/jobs/view/software-en...
